In [1]:
file_path = "C:/Users/yashm/Downloads/Dataset/loadedmodels/model.h5"
import h5py
# Create a new HDF5 file
with h5py.File(file_path, "w") as f:
    passer

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing import image
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, Callback
from tensorflow.keras.applications import ResNet50
from keras.callbacks import CSVLogger

# Load the ResNet50 model without top layers
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Create a new Sequential model
new_model = Sequential()

# Add the modified ResNet50 layers to the new model
new_model.add(base_model)  # Add the entire base_model as a layer

# Add a new FC layer with softmax activation
new_model.add(Flatten())
new_model.add(Dense(10, activation='softmax'))

# Freeze layers in base_model
for layer in base_model.layers:
    layer.trainable = False

# Compile the new model
new_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Display the summary
new_model.summary()

#DATA_FOLDER = "C:/Users/yashm/Downloads/Dataset"
TRAIN_PATH =  "Training/training-20240220T181906Z-001/training"
# Path for training data
VALID_PATH =  "Validation/validation-20240221T161041Z-001/validation"

# Define data generator to preprocess input data and create train and valid generators
batch_size = 10
datagen = image.ImageDataGenerator(preprocessing_function=tf.keras.applications.resnet50.preprocess_input)

# Create the generator for training data
train_generator = datagen.flow_from_directory(
    TRAIN_PATH,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Create the generator for validation data
valid_generator = datagen.flow_from_directory(
    VALID_PATH,
    target_size=(224, 224),
    batch_size=batch_size,
    class_mode='categorical'
)

# Define a custom callback to print best validation accuracy and epoch after every epoch
class PrintBestValidationAccuracy(Callback):
    def __init__(self):
        super(PrintBestValidationAccuracy, self).__init__()
        self.best_val_accuracy = -1
        self.best_epoch = -1

    def on_epoch_end(self, epoch, logs=None):
        val_accuracy = logs.get('val_accuracy')
        if val_accuracy is not None and val_accuracy > self.best_val_accuracy:
            self.best_val_accuracy = val_accuracy
            self.best_epoch = epoch + 1
        print(f"Best Validation Accuracy: {self.best_val_accuracy:.4f}, Achieved at Epoch: {self.best_epoch}")

# Define callbacks
early_stopping = EarlyStopping(monitor='val_loss',patience=20, restore_best_weights=True)
checkpoint = ModelCheckpoint("modelresnet.h5", 
                             monitor='val_accuracy', 
                             save_best_only=True, 
                             mode='max', 
                             verbose=1)
csv_logger = CSVLogger("epochinforesnet50.csv", separator=',', append=True)
print_best_val_accuracy_callback = PrintBestValidationAccuracy()

# Train the model for 200 epochs with early stopping
history = new_model.fit(
    train_generator,
    validation_data=valid_generator,
    epochs=200,
    steps_per_epoch=len(train_generator),
    validation_steps=len(valid_generator),
    callbacks=[early_stopping, checkpoint, csv_logger, print_best_val_accuracy_callback],
    initial_epoch=0
)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 resnet50 (Functional)       (None, 7, 7, 2048)        23587712  
                                                                 
 flatten (Flatten)           (None, 100352)            0         
                                                                 
 dense (Dense)               (None, 10)                1003530   
                                                                 
Total params: 24591242 (93.81 MB)
Trainable params: 1003530 (3.83 MB)
Non-trainable params: 23587712 (89.98 MB)
_________________________________________________________________
Found 16986 images belonging to 10 classes.
Found 4249 images belonging to 10 classes.
Epoch 3/100
1699/1699 [==============================] - ETA: 0s - loss: 8.0926 - accuracy: 0.8400
Epoch 3: val_accuracy improved from -inf to 0.86373, saving model to C:/Users/yashm/Down

C:\Users\yashm\anaconda3\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Best Validation Accuracy: 0.8637, Achieved at Epoch: 3
1699/1699 [==============================] - 936s 550ms/step - loss: 8.0926 - accuracy: 0.8400 - val_loss: 9.2214 - val_accuracy: 0.8637
Epoch 4/100
1699/1699 [==============================] - ETA: 0s - loss: 3.5620 - accuracy: 0.9354
Epoch 4: val_accuracy improved from 0.86373 to 0.89033, saving model to C:/Users/yashm/Downloads/Dataset/loadedmodels\model.h5
Best Validation Accuracy: 0.8903, Achieved at Epoch: 4
1699/1699 [==============================] - 881s 519ms/step - loss: 3.5620 - accuracy: 0.9354 - val_loss: 9.9253 - val_accuracy: 0.8903
Epoch 5/100
1699/1699 [==============================] - ETA: 0s - loss: 2.2495 - accuracy: 0.9599
Epoch 5: val_accuracy improved from 0.89033 to 0.90539, saving model to C:/Users/yashm/Downloads/Dataset/loadedmodels\model.h5
Best Validation Accuracy: 0.9054, Achieved at Epoch: 5
1699/1699 [==============================] - 848s 499ms/step - loss: 2.2495 - accuracy: 0.9599 - val_loss: 9.

KeyboardInterrupt: 

In [ ]:
import zipfile
with zipfile.ZipFile('Training.zip','r') as zip_ref:
    zip_ref.extractall()
    
import zipfile
with zipfile.ZipFile('Validation.zip','r') as zip_ref:
    zip_ref.extractall()